In [ ]:
import pandas as pd
!pip install transformers
import json
from transformers import AutoTokenizer

In [ ]:
import json

# Load the JSONL dataset
data_file = "/content/super_scotus_sample.jsonl"  # path to the dataset file
cases = {}
with open(data_file, "r") as f:
    for line in f:
        case = json.loads(line)
        case_id = case.get("case_id") or case.get("id")  # use appropriate ID field
        # Separate Justia opinion sections and convos (transcript)
        justia_sections = case.get("justia_sections", {})  # e.g. {"Syllabus": "...", "Opinion": "...", ...}
        convos = case.get("convos", {})
        cases[case_id] = {
            "justia_sections": justia_sections,
            "convos": convos
        }

print(f"Loaded {len(cases)} cases from Super-SCOTUS dataset.")
# Example: Check structure for one case
example_id, example_case = next(iter(cases.items()))
print("Case ID:", example_id)
print("Justia sections:", list(example_case["justia_sections"].keys()))
# If 'convos' is a dict of parallel lists (ids, speakers, text, etc.), convert to list of utterance dicts:
convos = example_case["convos"]
if isinstance(convos, dict) and "ids" in convos:
    utterances = []
    ids = convos.get("ids", [])
    texts = convos.get("text") or convos.get("texts") or convos.get("utterances")
    speakers = convos.get("speakers") or convos.get("speaker")
    sides = convos.get("sides") or convos.get("side")
    speaker_types = convos.get("speaker_type") or convos.get("speaker_types")
    # Combine parallel lists into structured utterances
    for i, u_id in enumerate(ids):
        utterance = {
            "id": u_id,
            "text": texts[i] if texts else "",
            "speaker": speakers[i] if speakers else None,
            "side": sides[i] if sides else None,
            "speaker_type": speaker_types[i] if speaker_types else None
        }
        utterances.append(utterance)
    cases[case_id]["convos"] = utterances
elif isinstance(convos, list):
    # Already a list of utterance dicts
    cases[case_id]["convos"] = convos

print("First 2 transcript utterances:", cases[example_id]["convos"][:2])


Loaded 10 cases from Super-SCOTUS dataset.
Case ID: 1955_71
Justia sections: ['Syllabus', 'Case']


KeyError: 0

In [11]:
import json

# Load the JSONL dataset
data_file = "/content/super_scotus_sample_clean.jsonl"  # path to the dataset file
cases = {}

with open(data_file, "r") as f:
    for line in f:
        case = json.loads(line)
        case_id = case.get("case_id") or case.get("id")  # use appropriate ID field
        justia_sections = case.get("justia_sections", {})  # e.g. {"Syllabus": "...", "Opinion": "...", ...}
        convos = case.get("convos", {})

        # Begin parsing the convos
        utterances = []

        if isinstance(convos, dict) and "ids" in convos:
            ids = convos.get("ids", [])
            texts = convos.get("text") or convos.get("texts") or []
            speaker_map = convos.get("speaker", {})  # This is a lookup dictionary
            sides_map = speaker_map  # sides are stored inside speaker metadata
            types_map = speaker_map  # same for types

            for i, speaker_id in enumerate(ids):
                u_text = texts[i] if i < len(texts) else ""
                speaker_meta = speaker_map.get(speaker_id, {}) if speaker_id else {}

                utterance = {
                    "id": speaker_id,
                    "text": u_text,
                    "speaker": speaker_id,
                    "side": speaker_meta.get("side"),
                    "speaker_type": speaker_meta.get("type")
                }
                utterances.append(utterance)

        elif isinstance(convos, list):
            # Already a list of utterance dicts
            utterances = convos

        # Store the parsed case
        cases[case_id] = {
            "justia_sections": justia_sections,
            "convos": utterances
        }

print(f"Loaded {len(cases)} cases from Super-SCOTUS dataset.")

# Example: Check structure for one case
example_id, example_case = next(iter(cases.items()))
print("Case ID:", example_id)
print("Justia sections:", list(example_case["justia_sections"].keys()))
print("First 2 transcript utterances:", example_case["convos"][:2])


Loaded 6733 cases from Super-SCOTUS dataset.
Case ID: 1955_71
Justia sections: ['Syllabus', 'Case']
First 2 transcript utterances: [{'id': '13127', 'text': '', 'speaker': '13127', 'side': None, 'speaker_type': None}]


In [15]:
import json

# Load the JSONL dataset
data_file = "/content/super_scotus_sample_clean.jsonl"  # path to the dataset file
cases = {}

with open(data_file, "r") as f:
    for line in f:
        case = json.loads(line)
        case_id = case.get("case_id") or case.get("id")  # use appropriate ID field
        justia_sections = case.get("justia_sections", {})  # e.g. {"Syllabus": "...", "Opinion": "...", ...}
        convos = case.get("convos", {})

        # Begin parsing the convos
        utterances = []
        speaker_map = convos.get("speaker", {})  # speaker metadata (lookup by speaker_id)

        # ✅ Case 1: Proper "utterances" field (nested list of dicts)
        if isinstance(convos, dict) and "utterances" in convos:
            outer = convos["utterances"]
            if (isinstance(outer, list) and len(outer) > 0 and isinstance(outer[0], list)
                    and all(isinstance(u, dict) for u in outer[0])):

                for utt in outer[0]:  # Flatten and parse
                    speaker_id = utt.get("speaker_id")
                    speaker_meta = speaker_map.get(speaker_id, {}) if speaker_id else {}

                    utterances.append({
                        "id": utt.get("id"),  # Some may not have an ID
                        "text": utt.get("text", ""),
                        "speaker": speaker_id,
                        "side": speaker_meta.get("side"),
                        "speaker_type": speaker_meta.get("type")
                    })
            else:
                print(f"Warning: Unexpected 'utterances' format in case {case_id}. Skipping it.")

        # ✅ Case 2: Parallel list fallback
        elif isinstance(convos, dict) and "ids" in convos:
            ids = convos.get("ids", [])
            texts = convos.get("text") or convos.get("texts") or []

            for i, speaker_id in enumerate(ids):
                u_text = texts[i] if i < len(texts) else ""
                speaker_meta = speaker_map.get(speaker_id, {}) if speaker_id else {}

                utterances.append({
                    "id": speaker_id,
                    "text": u_text,
                    "speaker": speaker_id,
                    "side": speaker_meta.get("side"),
                    "speaker_type": speaker_meta.get("type")
                })

        # ✅ Case 3: Already a flat list of utterance dicts
        elif isinstance(convos, list):
            utterances = convos

        # Store the parsed case
        cases[case_id] = {
            "justia_sections": justia_sections,
            "convos": utterances
        }

print(f"Loaded {len(cases)} cases from Super-SCOTUS dataset.")

# Example: Check structure for one case
example_id, example_case = next(iter(cases.items()))
print("Case ID:", example_id)
print("Justia sections:", list(example_case["justia_sections"].keys()))
print("First 2 transcript utterances:", example_case["convos"][:2])


Loaded 6733 cases from Super-SCOTUS dataset.
Case ID: 1955_71
Justia sections: ['Syllabus', 'Case']
First 2 transcript utterances: [{'id': None, 'text': 'Number 71, Lonnie Affronti versus United States of America.\nMr. Murphy.', 'speaker': 'j__earl_warren', 'side': None, 'speaker_type': 'J'}, {'id': None, 'text': 'May it please the Court.\nWe are here by writ of certiorari to the Eighth Circuit.\nThere is one question to be decided in this case, decided carefully.\nUpon sentence to consecutive sentences or terms by a District Court.\nThe defending pattern started the service of a first sentence.\nThus, the District Court thereafter have jurisdiction to suspend the execution of the remaining sentences and place the defendant on probation.', 'speaker': 'harry_f_murphy', 'side': 1, 'speaker_type': 'A'}]


In [9]:
import json

# Load the JSONL dataset
data_file = "/content/super_scotus_sample_clean.jsonl"  # path to the dataset file
cases = {}

with open(data_file, "r") as f:
    for line in f:
        try:
            case = json.loads(line)
        except json.JSONDecodeError as e:
                print(f"Skipping malformed line: {e}")
                continue
        case_id = case.get("case_id") or case.get("id")  # use appropriate ID field
        justia_sections = case.get("justia_sections", {})  # e.g. {"Syllabus": "...", "Opinion": "...", ...}
        convos = case.get("convos", {})

        # Begin parsing the convos
        utterances = []
        speaker_map = convos.get("speaker", {})  # speaker metadata (lookup by speaker_id)

        # ✅ Case 1: Proper "utterances" field (nested list of dicts)
        if isinstance(convos, dict) and "utterances" in convos:
            outer = convos["utterances"]
            if (isinstance(outer, list) and len(outer) > 0 and isinstance(outer[0], list)
                    and all(isinstance(u, dict) for u in outer[0])):

                for utt in outer[0]:  # Flatten and parse
                    utt_id = utt.get("id")
                    speaker_id = utt.get("speaker_id")
                    text = utt.get("text", "")

                    speaker_meta = speaker_map.get(speaker_id, {}) if speaker_id else {}
                    speaker_type = speaker_meta.get("type")

                    # Debug this
                    if speaker_type is None:
                        print(f"[DEBUG] Missing speaker_type for: speaker_id={speaker_id}, utt_text={text[:50]}")

                    utterances.append({
                         "id": utt_id,
                         "text": text,
                         "speaker": speaker_id,
                         "side": speaker_meta.get("side"),
                         "speaker_type": speaker_type
                    })

            else:
                print(f"Warning: Unexpected 'utterances' format in case {case_id}. Skipping it.")

        # ✅ Case 2: Parallel list fallback
        elif isinstance(convos, dict) and "ids" in convos:
            ids = convos.get("ids", [])
            texts = convos.get("text") or convos.get("texts") or []

            for i, speaker_id in enumerate(ids):
                u_text = texts[i] if i < len(texts) else ""
                speaker_meta = speaker_map.get(speaker_id, {}) if speaker_id else {}

                utterances.append({
                    "id": speaker_id,
                    "text": u_text,
                    "speaker": speaker_id,
                    "side": speaker_meta.get("side"),
                    "speaker_type": speaker_meta.get("type")
                })

        # ✅ Case 3: Already a flat list of utterance dicts
        elif isinstance(convos, list):
            utterances = convos

        # Store the parsed case
        cases[case_id] = {
            "justia_sections": justia_sections,
            "convos": utterances
        }

print(f"Loaded {len(cases)} cases from Super-SCOTUS dataset.")

# Example: Check structure for one case
example_id, example_case = next(iter(cases.items()))
print("Case ID:", example_id)
print("Justia sections:", list(example_case["justia_sections"].keys()))
print("First 2 transcript utterances:", example_case["convos"][:2])


Loaded 6733 cases from Super-SCOTUS dataset.
Case ID: 1955_71
Justia sections: ['Syllabus', 'Case']
First 2 transcript utterances: [{'id': None, 'text': 'Number 71, Lonnie Affronti versus United States of America.\nMr. Murphy.', 'speaker': 'j__earl_warren', 'side': None, 'speaker_type': 'J'}, {'id': None, 'text': 'May it please the Court.\nWe are here by writ of certiorari to the Eighth Circuit.\nThere is one question to be decided in this case, decided carefully.\nUpon sentence to consecutive sentences or terms by a District Court.\nThe defending pattern started the service of a first sentence.\nThus, the District Court thereafter have jurisdiction to suspend the execution of the remaining sentences and place the defendant on probation.', 'speaker': 'harry_f_murphy', 'side': 1, 'speaker_type': 'A'}]


In [12]:
!pip install langchain tiktoken  # ensure LangChain is installed for text splitting

from langchain.text_splitter import RecursiveCharacterTextSplitter

# Initialize a text splitter (e.g., aiming for ~300 tokens, with some overlap)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=100)  # chunk_size in characters (approx tokens)

justia_chunks = []  # will hold LangChain Document objects for each chunk
from langchain.docstore.document import Document

for case_id, case_data in cases.items():
    sections = case_data["justia_sections"]
    if not sections:
        continue  # skip if no Justia text
    for section_name, section_text in sections.items():
        # Determine section type and justice (if section_name contains a Justice's name for concurrences/dissents)
        sec_type = section_name
        justice = None
        # Example: section_name might be "Opinion" or "Opinion of Justice X", or "Dissenting - Justice Y"
        if "Concurring" in section_name or "Dissent" in section_name:
            sec_type = "Concurring" if "Concurring" in section_name else "Dissenting"
            # Extract justice name if present
            # e.g. "Concurring Opinion by Justice Doe" or "Justice Doe, dissenting"
            words = section_name.split()
            for w in words:
                if w.capitalize() == "Justice" or w.endswith("J.") or w.endswith("C.J."):
                    # The next word might be last name of Justice
                    # Simple heuristic: find index of "Justice" and take next word
                    try:
                        idx = words.index(w)
                        justice = words[idx+1].strip(",")
                    except:
                        justice = None
            # If justice still None, it might be something like "Justice Doe, dissenting"
            if justice is None and "Justice" in section_text:
                # fallback: find first occurrence of "Justice" in text
                start = section_text.find("Justice")
                if start != -1:
                    justice = section_text[start:section_text.find(" ", start+8)]
        elif "Opinion" in section_name and "Justice" in section_name:
            sec_type = "Opinion"
            # e.g. "Opinion of Justice X"
            try:
                justice_idx = section_name.split().index("Justice")
                justice = section_name.split()[justice_idx+1]
            except:
                justice = None
        else:
            # e.g. "Syllabus" or generic "Opinion"
            sec_type = section_name

        # Split section text into chunks
        for chunk in text_splitter.split_text(section_text):
            doc = Document(
                page_content=chunk,
                metadata={
                    "case_id": case_id,
                    "section_type": sec_type,
                    "justice": justice
                }
            )
            justia_chunks.append(doc)

print(f"Total Justia chunks: {len(justia_chunks)} (from {len(cases)} cases)")
print("Sample chunk metadata:", justia_chunks[0].metadata)
print("Sample chunk text snippet:", justia_chunks[0].page_content[:100], "...")


Total Justia chunks: 353627 (from 6733 cases)
Sample chunk metadata: {'case_id': '1955_71', 'section_type': 'Syllabus', 'justice': None}
Sample chunk text snippet: U.S. Supreme CourtAffronti v. United States, 350 U.S. 79 (1955)Affronti v. United StatesNo. 71Argued ...


In [16]:
!pip install nltk
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize

convos_chunks = []  # list of Document chunks for transcripts

for case_id, case_data in cases.items():
    utterances = case_data.get("convos")
    if not utterances:
        continue
    # Determine phase order by looking at sides of advocate speakers
    current_phase = None  # "Petitioner", "Respondent", or "Rebuttal"
    current_side = None   # "petitioner" or "respondent" (from data)
    current_chunk_text = ""
    current_chunk_utts = []

    for utt in utterances:
        speaker_type = utt.get("speaker_type")
        side = utt.get("side")  # e.g., "petitioner" or "respondent" or None for justices
        text = utt.get("text", "").strip()
        if not text:
            continue  # skip empty utterances if any

        # Determine if a phase switch should occur
        if speaker_type == "A" or speaker_type == "Advocate":  # an attorney (advocate) turn
            # Map side to phase name (capitalize first letter)
            if side:
                side_name = side.capitalize()  # "Petitioner" or "Respondent"
            else:
                # If side not given, infer from context or speaker name (not covered here)
                side_name = current_phase or "Petitioner"
        else:
            # A Justice speaking does not change the phase; they are asking questions.
            side_name = current_phase

        # Initialize phase if not set
        if current_phase is None:
            current_phase = side_name

        # If the side/phase has changed (from Petitioner to Respondent or vice versa),
        # or if we explicitly detect a "Rebuttal" (Petitioner speaking again after Respondent):
        if side_name and current_phase and side_name != current_phase:
            # Finish the current phase chunk (if any text accumulated)
            if current_chunk_text:
                convos_chunks.append(Document(
                    page_content=current_chunk_text.strip(),
                    metadata={
                        "case_id": case_id,
                        "phase": current_phase,
                        "utterance_ids": current_chunk_utts
                    }
                ))
            # Reset for new phase
            current_phase = side_name
            current_chunk_text = ""
            current_chunk_utts = []

        # Add the current utterance to the chunk
        # Check if adding this utterance would exceed token limit (~300-500 tokens)
        tokens = word_tokenize(text)
        if len(word_tokenize(current_chunk_text)) + len(tokens) > 400:
            # If current chunk is already non-empty, close it and start a new chunk
            if current_chunk_text:
                convos_chunks.append(Document(
                    page_content=current_chunk_text.strip(),
                    metadata={
                        "case_id": case_id,
                        "phase": current_phase,
                        "utterance_ids": current_chunk_utts
                    }
                ))
            # Reset chunk content
            current_chunk_text = ""
            current_chunk_utts = []
        # Append this utterance
        current_chunk_text += " " + text
        current_chunk_utts.append(utt.get("id"))

    # Add the final chunk for this case and phase if exists
    if current_chunk_text:
        convos_chunks.append(Document(
            page_content=current_chunk_text.strip(),
            metadata={
                "case_id": case_id,
                "phase": current_phase,
                "utterance_ids": current_chunk_utts
            }
        ))

print(f"Total transcript chunks: {len(convos_chunks)}")
print("Sample transcript chunk metadata:", convos_chunks[0].metadata)
print("Sample chunk text snippet:", convos_chunks[0].page_content[:100], "...")


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


LookupError: 
**********************************************************************
  Resource [93mpunkt_tab[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt_tab')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt_tab/english/[0m

  Searched in:
    - '/root/nltk_data'
    - '/usr/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************


In [17]:
!pip install nltk
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from langchain.schema import Document  # Ensure this import if not already included

# You can also debug available speaker types with:
speaker_types_seen = set()
for case in cases.values():
    for utt in case.get("convos", []):
        if utt.get("speaker_type"):
            speaker_types_seen.add(utt["speaker_type"].lower())

print("Detected speaker types in dataset:", speaker_types_seen)

# Define which speaker types should be treated as 'advocates'
ADVOCATE_TYPES = {"a", "advocate", "attorney", "counsel"}

convos_chunks = []  # list of Document chunks for transcripts

for case_id, case_data in cases.items():
    utterances = case_data.get("convos")
    if not utterances:
        continue

    current_phase = None  # "Petitioner", "Respondent", or "Rebuttal"
    current_chunk_text = ""
    current_chunk_utts = []

    for utt in utterances:
        speaker_type = utt.get("speaker_type")
        side = utt.get("side")  # e.g., "petitioner", "respondent", or None
        text = utt.get("text", "").strip()
        if not text:
            continue  # skip empty utterances

        # Normalize speaker_type string
        speaker_type_normalized = speaker_type.lower() if speaker_type else None

        # Determine phase from speaker_type and side
        if speaker_type_normalized in ADVOCATE_TYPES:
            side_name = side.capitalize() if side else (current_phase or "Petitioner")
        else:
            side_name = current_phase  # Justices don't shift phase

        # Initialize current phase if needed
        if current_phase is None:
            current_phase = side_name

        # Detect phase switch
        if side_name and current_phase and side_name != current_phase:
            if current_chunk_text:
                convos_chunks.append(Document(
                    page_content=current_chunk_text.strip(),
                    metadata={
                        "case_id": case_id,
                        "phase": current_phase,
                        "utterance_ids": current_chunk_utts
                    }
                ))
            # Reset for new phase
            current_phase = side_name
            current_chunk_text = ""
            current_chunk_utts = []

        # Estimate tokens and split if needed
        tokens = word_tokenize(text)
        if len(word_tokenize(current_chunk_text)) + len(tokens) > 400:
            if current_chunk_text:
                convos_chunks.append(Document(
                    page_content=current_chunk_text.strip(),
                    metadata={
                        "case_id": case_id,
                        "phase": current_phase,
                        "utterance_ids": current_chunk_utts
                    }
                ))
            current_chunk_text = ""
            current_chunk_utts = []

        # Accumulate this utterance
        current_chunk_text += " " + text
        current_chunk_utts.append(utt.get("id"))

    # Add final chunk if it exists
    if current_chunk_text:
        convos_chunks.append(Document(
            page_content=current_chunk_text.strip(),
            metadata={
                "case_id": case_id,
                "phase": current_phase,
                "utterance_ids": current_chunk_utts
            }
        ))

# ✅ Safe printing
print(f"\nTotal transcript chunks: {len(convos_chunks)}")
if convos_chunks:
    print("Sample transcript chunk metadata:", convos_chunks[0].metadata)
    print("Sample chunk text snippet:", convos_chunks[0].page_content[:100], "...")
else:
    print("⚠️ No transcript chunks were generated. Check if any utterances matched speaker_type filters.")


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Detected speaker types in dataset: {'u', 'j', 'a'}


LookupError: 
**********************************************************************
  Resource [93mpunkt_tab[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt_tab')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt_tab/english/[0m

  Searched in:
    - '/root/nltk_data'
    - '/usr/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************


In [18]:
# Ensure NLTK is installed
!pip install nltk --quiet

import nltk
import shutil
from nltk.tokenize import word_tokenize
from langchain.schema import Document  # Ensure this import if not already included

# 🔁 Step 1: Clean and redownload tokenizer data to fix punkt_tab issue
shutil.rmtree('/root/nltk_data', ignore_errors=True)  # Optional but ensures clean state
nltk.download('punkt')  # Redownload tokenizer model

# 🔍 Debug available speaker types
speaker_types_seen = set()
for case in cases.values():
    for utt in case.get("convos", []):
        if utt.get("speaker_type"):
            speaker_types_seen.add(utt["speaker_type"].lower())

print("Detected speaker types in dataset:", speaker_types_seen)

# ⚖️ Define which speaker types are considered 'advocates'
ADVOCATE_TYPES = {"a", "advocate", "attorney", "counsel"}

# 📄 Chunk container
convos_chunks = []

for case_id, case_data in cases.items():
    utterances = case_data.get("convos")
    if not utterances:
        continue

    current_phase = None  # e.g., "Petitioner", "Respondent"
    current_chunk_text = ""
    current_chunk_utts = []

    for utt in utterances:
        speaker_type = utt.get("speaker_type")
        side = utt.get("side")
        text = utt.get("text", "").strip()
        if not text:
            continue

        # Normalize speaker_type
        speaker_type_normalized = speaker_type.lower() if speaker_type else None

        # Determine phase based on advocate role
        if speaker_type_normalized in ADVOCATE_TYPES:
            side_name = side.capitalize() if side else (current_phase or "Petitioner")
        else:
            side_name = current_phase

        # Initialize phase if not already set
        if current_phase is None:
            current_phase = side_name

        # Detect phase change and split
        if side_name and current_phase and side_name != current_phase:
            if current_chunk_text:
                convos_chunks.append(Document(
                    page_content=current_chunk_text.strip(),
                    metadata={
                        "case_id": case_id,
                        "phase": current_phase,
                        "utterance_ids": current_chunk_utts
                    }
                ))
            current_phase = side_name
            current_chunk_text = ""
            current_chunk_utts = []

        # Estimate tokens and split if needed
        tokens = word_tokenize(text)
        if len(word_tokenize(current_chunk_text)) + len(tokens) > 400:
            if current_chunk_text:
                convos_chunks.append(Document(
                    page_content=current_chunk_text.strip(),
                    metadata={
                        "case_id": case_id,
                        "phase": current_phase,
                        "utterance_ids": current_chunk_utts
                    }
                ))
            current_chunk_text = ""
            current_chunk_utts = []

        # Add utterance to chunk
        current_chunk_text += " " + text
        current_chunk_utts.append(utt.get("id"))

    # Final chunk flush
    if current_chunk_text:
        convos_chunks.append(Document(
            page_content=current_chunk_text.strip(),
            metadata={
                "case_id": case_id,
                "phase": current_phase,
                "utterance_ids": current_chunk_utts
            }
        ))

# ✅ Report
print(f"\nTotal transcript chunks: {len(convos_chunks)}")
if convos_chunks:
    print("Sample transcript chunk metadata:", convos_chunks[0].metadata)
    print("Sample chunk text snippet:", convos_chunks[0].page_content[:100], "...")
else:
    print("⚠️ No transcript chunks were generated. Check if any utterances matched speaker_type filters.")


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Detected speaker types in dataset: {'u', 'j', 'a'}


LookupError: 
**********************************************************************
  Resource [93mpunkt_tab[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt_tab')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt_tab/english/[0m

  Searched in:
    - '/root/nltk_data'
    - '/usr/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************


In [20]:
# --- Setup ---
!pip install nltk langchain --quiet

import nltk
import shutil
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document

# 🔁 Ensure clean download of tokenizer models
shutil.rmtree('/root/nltk_data', ignore_errors=True)
nltk.download('punkt')

# --- Parameters ---
ADVOCATE_TYPES = {"a", "advocate", "attorney", "counsel"}
MAX_CHUNK_SIZE = 1000  # characters (can also use token-based splitter)

# --- LangChain's TextSplitter ---
splitter = RecursiveCharacterTextSplitter(
    chunk_size=MAX_CHUNK_SIZE,
    chunk_overlap=100,
    separators=["\n\n", "\n", ".", "!", "?", " ", ""]
)

# --- Phase-Level Collection ---
phase_blocks = []

for case_id, case_data in cases.items():
    utterances = case_data.get("convos")
    if not utterances:
        continue

    current_phase = None
    current_text = ""
    current_utts = []

    for utt in utterances:
        speaker_type = utt.get("speaker_type", "").lower()
        side = utt.get("side")
        text = utt.get("text", "").strip()
        if not text:
            continue

        side_name = str(side).capitalize() if speaker_type in ADVOCATE_TYPES and side is not None else current_phase


        if current_phase is None:
            current_phase = side_name

        # Phase shift
        if side_name != current_phase and current_text:
            phase_blocks.append(Document(
                page_content=current_text.strip(),
                metadata={"case_id": case_id, "phase": current_phase, "utterance_ids": current_utts}
            ))
            current_phase = side_name
            current_text = ""
            current_utts = []

        current_text += " " + text
        current_utts.append(utt.get("id"))

    if current_text:
        phase_blocks.append(Document(
            page_content=current_text.strip(),
            metadata={"case_id": case_id, "phase": current_phase, "utterance_ids": current_utts}
        ))

# --- Now use LangChain to split all phase blocks into chunks ---
convos_chunks = []
for doc in phase_blocks:
    chunks = splitter.split_documents([doc])
    convos_chunks.extend(chunks)

# --- Report ---
print(f"\nTotal transcript chunks: {len(convos_chunks)}")
if convos_chunks:
    print("Sample transcript chunk metadata:", convos_chunks[0].metadata)
    print("Sample chunk text snippet:", convos_chunks[0].page_content[:100], "...")
else:
    print("⚠️ No transcript chunks were generated.")


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!



Total transcript chunks: 443294
Sample transcript chunk metadata: {'case_id': '1955_71', 'phase': '1', 'utterance_ids': [None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None]}
Sample chunk text snippet: Number 71, Lonnie Affronti versus United States of America.
Mr. Murphy. May it please the Court.
We  ...
